In [39]:
import pandas as pd
import os

ot_odr_filename = os.path.join(".", "OT_ODR.csv.bz2")
ot_odr_df = pd.read_csv(ot_odr_filename, compression="bz2", sep=";")
longueur_ot_odr_df = len(ot_odr_df)

equipements_filename = os.path.join(".", 'EQUIPEMENTS.csv')
equipements_df = pd.read_csv(equipements_filename, sep=";")


In [2]:
equipements_df.tail()


,EQU_ID,MODELE,CONSTRUCTEUR,MOTEUR
4882,E00271009,MD023,C007,MT021
4883,E00341892,MD063,C007,MT023
4884,E00331551,MD005,C000,MT040
4885,E00373890,MD021,C011,MT0-1
4886,E00365136,MD032,C006,MT016


In [3]:
ot_odr_df.tail()

,OT_ID,ODR_ID,ODR_LIBELLE,TYPE_TRAVAIL,DUREE_TRAVAIL,SYSTEM_N1,SYSTEM_N2,SYSTEM_N3,EQU_ID,DATE_OT,KILOMETRAGE,SIG_ORGANE,SIG_CONTEXTE,SIG_OBS,LIGNE
506553,OT000405952,OM000506539,REMPLACEMENT D'UN FEU DE RECUL,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00040793,2019-09-30 20:38:09,296005.373397,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/EXTERIEUR/DROIT,CASSE,L0283
506554,OT000405953,OM000506546,REMPLACEMENT D'UN ECLAIRAGE EXT,MECANIQUE,0.75,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00274690,2019-09-30 21:21:20,153050.080050,ECLAIRAGE FEUX EXTERIEURS,ARRIERE/LATERAL,CASSE,L0116
506555,OT000405954,OM000506536,REMPLACEMENT D'UN PASSE SANS CONTACT,EQUIPEMENT EMBARQUE,0.03,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,PASSE SANS CONTACT,E00256452,2019-09-30 21:39:29,175063.182439,AVTT,AVANT/PORTE,INTERMITTENT,L0134
506556,OT000405955,OM000506545,REMPLACEMENT D'UNE LAMPE DE FEU DE GABARIT,ELECTRICITE,0.04,EQUIPEMENT ELECTRIQUE,ECLAIRAGE-SIGNALISATION,ECLAIRAGE-SIGNALISATION EXT,E00006122,2019-09-30 21:55:28,437053.614263,ECLAIRAGE FEUX EXTERIEURS,LATERAL/HAUT/GAUCHE,NE FONCTIONNE PAS,L0270
506557,OT000405956,OM000506547,REMPLACEMENT DE LA CASSETTE ENCREUSE D'UN VALI...,EQUIPEMENT EMBARQUE,0.03,EQUIPEMENT EMBARQUE,TELEBILLETIQUE,VALIDEUR MAGNETIQUE,E00315330,2019-09-30 23:43:42,83269.095630,AVTT,POSTE CONDUITE/AVANT,NE FONCTIONNE PAS,L0047


In [4]:
ot_odr_df['SIG_ORGANE'].value_counts()

SIG_ORGANE
ECLAIRAGE FEUX EXTERIEURS    189559
CLIGNOTANTS/ DETRESSE         38986
PARE-CHOCS                    33191
RETROVISEUR                   25005
PORTE                         23043
                              ...  
TACHYGRAPHE                       2
LUBRIMATIC                        1
COLMATAGE.F.AIR                   1
PILE BTSAM                        1
ISSUE SEC.                        1
Name: count, Length: 116, dtype: int64

### PyAgrum

In [24]:
!pip install pyAgrum
import pyAgrum as gum

rb_projet =  gum.BayesNet("Réseau bayésien")

## Création du noeud de SIG_ORGANE

In [26]:

SIG_ORGANE_ELEMENT = ot_odr_df['SIG_ORGANE'].value_counts()

va_SIG_ORGANE = gum.LabelizedVariable("SIG_ORGANE", "SIG_ORGANE", len(SIG_ORGANE_ELEMENT))

i = 0
for liste in ot_odr_df['SIG_ORGANE'].unique():
    va_SIG_ORGANE.changeLabel(i,liste)
    i += 1


In [27]:

SIG_SYSTEM_N1 = ot_odr_df['SYSTEM_N1'].value_counts()

va_SYSTEM_N1 = gum.LabelizedVariable("SYSTEM_N1", "SYSTEM_N1", len(SIG_SYSTEM_N1))

i = 0
for liste in ot_odr_df['SYSTEM_N1'].unique():
    va_SYSTEM_N1.changeLabel(i,liste)
    i += 1


In [28]:
for va in [va_SIG_ORGANE,va_SYSTEM_N1]:
    rb_projet.add(va)

In [29]:
rb_projet.addArc("SIG_ORGANE","SYSTEM_N1")

## Probabilité pour SIG_ORGANE

In [41]:
count_SIG_ORGANE = []

for liste in ot_odr_df['SIG_ORGANE'].unique():
    count_SIG_ORGANE.append(len(ot_odr_df.loc[ot_odr_df['SIG_ORGANE'] == liste])/longueur_ot_odr_df)

print(count_SIG_ORGANE)
rb_projet.cpt("SIG_ORGANE")[:] = count_SIG_ORGANE



[0.0024064371700772665, 0.0008607109156305892, 0.016896386988261958, 0.06552260550618093, 0.0007935912570722405, 0.0014509690894231263, 0.3742098634312359, 0.03726522925311613, 0.0029137828244741966, 0.0454893615341185, 0.013929303258462013, 0.03490024834273667, 0.0012535583289573948, 0.0039620339625472305, 0.010040311277287103, 0.011418238385337908, 0.022380457913999976, 0.07696255907517006, 0.00766743393648901, 6.317144334903408e-05, 0.013575937997228353, 0.0042186679511526815, 0.0003020384635125691, 0.0024735568286356154, 0.0033658534659407216, 0.0010087689859798877, 0.049362560654456156, 0.03116326264712037, 0.0030835560784747253, 0.0007225233833045772, 0.003211873072777451, 0.00047378582511775554, 0.004165367045826934, 0.0001480580703492986, 0.0008745296688631904, 0.00346258473856893, 0.005014233315829579, 0.0021438808586578435, 0.0007639796430023807, 0.0013818753232601204, 0.0008488662700026453, 0.0019839781426806012, 0.008832157423236826, 0.0010778627521428937, 0.000363235799256